In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision


print(torch.cuda.is_available())
if torch.cuda.is_available():
    print("Cuda is Availabe")
else:
    print("Cuda Can't be found")

# Just the device selection options
device = (
    "cuda:4" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

True
Cuda is Availabe
Using cuda:4 device


In [3]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.act2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.act3 = nn.Tanh()

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1*1*120, 84)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(84, 10)
        
    def forward(self, x):
        # input 1x28x28, output 6x28x28
        x = self.act1(self.conv1(x))
        # input 6x28x28, output 6x14x14
        x = self.pool1(x)
        # input 6x14x14, output 16x10x10
        x = self.act2(self.conv2(x))
        # input 16x10x10, output 16x5x5
        x = self.pool2(x)
        # input 16x5x5, output 120x1x1
        x = self.act3(self.conv3(x))
        # input 120x1x1, output 84
        x = self.act4(self.fc1(self.flat(x)))
        # input 84, output 10
        x = self.fc2(x)
        return x

## 

In [4]:
import os
from torch.utils.data import Dataset
from PIL import Image
import json
class ImageNetKaggle(Dataset):
    def __init__(self, root, split, transform=None):
        self.samples = []
        self.targets = []
        self.transform = transform
        self.syn_to_class = {}
        with open(os.path.join(root, "imagenet_class_index.json"), "rb") as f:
                    json_file = json.load(f)
                    for class_id, v in json_file.items():
                        self.syn_to_class[v[0]] = int(class_id)
        with open(os.path.join(root, "ILSVRC2012_val_labels.json"), "rb") as f:
                    self.val_to_syn = json.load(f)
        samples_dir = os.path.join(root, "", split)
        for entry in os.listdir(samples_dir):
            
            syn_id = self.val_to_syn[entry]
            target = self.syn_to_class[syn_id]
            sample_path = os.path.join(samples_dir, entry)
            self.samples.append(sample_path)
            self.targets.append(target)
    def __len__(self):
            return len(self.samples)
    def __getitem__(self, idx):
            x = Image.open(self.samples[idx]).convert("RGB")
            if self.transform:
                x = self.transform(x)
            return x, self.targets[idx]

In [5]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0,), (128,)),
])
train = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=100)
testloader = torch.utils.data.DataLoader(test, shuffle=True, batch_size=100)

100%|██████████| 9912422/9912422 [00:01<00:00, 5193941.03it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 17382076.89it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1391021.69it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9824924.58it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [9]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import torchvision
from tqdm import tqdm
import os

optimizer = None
loss_fn = None
BASE_DIR = os.getcwd()



def load_model(model_name):
    global model
    if model_name.lower() == 'resnet':
        model = torchvision.models.resnet50(weights="DEFAULT")
    elif model_name.lower() == 'alexnet':
        model =torchvision.models.alexnet(weights="DEFAULT")
    elif model_name.lower() =='googlenet':
        model=torchvision.models.googlenet(weights="DEFAULT")
    elif model_name.lower() =='vgg16':
        model = torchvision.models.vgg16(weights="DEFAULT")
    elif model_name.lower() =='lenet':
        global  optimizer, loss_fn
        print (os.path.curdir)
        model = LeNet5().to(device)
        optimizer = optim.Adam(model.parameters())
        loss_fn = nn.CrossEntropyLoss()
    
        MODEL_FILENAME=os.path.join(BASE_DIR, 'model.sav')
        if (os.path.isfile(MODEL_FILENAME) != True):
            __train__(model, 10)
            torch.save(model.state_dict(),MODEL_FILENAME)
            print (f'Model saved to {MODEL_FILENAME}')
        else:
            model.load_state_dict(torch.load(MODEL_FILENAME))
            model.to(device)
            model.eval()
    model.to(device)
    model.eval()   


imagenet_path = os.path.join(BASE_DIR, 'imagenet')

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
val_transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]
        )
dataset = ImageNetKaggle(imagenet_path, "val", val_transform)
dataloader = DataLoader(
            dataset,
            batch_size=64, # may need to reduce this depending on your GPU 
            num_workers=8, # may need to reduce this depending on your num of CPUs and RAM
            shuffle=False,
            drop_last=False,
            pin_memory=True
        )
def __test__ (model__,model_name)-> float:
    if model_name.lower() =='lenet':
        model__.eval()
        acc = 0
        count = 0
        for X_batch, y_batch in testloader:
            X_batch = X_batch.to(device); y_batch = y_batch.to(device)
            y_pred = model__(X_batch)
            acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
            count += len(y_batch)
        acc = acc / count
        acc__ = acc.cpu()
        return acc__.numpy()*100
    else:
        acc = 0
        total = 0
        with torch.no_grad():
            for x, y in tqdm(dataloader):
                x= x.to(device)
                y=y.to(device)
                y_pred = model(x)
                acc += (torch.argmax(y_pred, 1) == y).float().sum()
                total += len(y)
        acc=acc / total
        acc__=acc.cpu()
        return acc__.numpy()*100


    

In [10]:

import pandas as pd
import datetime, time,threading, psutil

class SystemStatsGatherer:
    def __init__(self,  interval=2):
        self.temp_dict = {}
        self.not_firstime = False
        self.diskio_fieldname = ['read_count', 'write_count','read_bytes', 'write_bytes', 'read_time', 'write_time', 'read_merged_count', 'write_merged_count', 'busy_time']
        self.diskio_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0]
        self.swap_mem_fieldname = ['total', 'used', 'free', 'percent', 'sin', 'sout']
        self.swap_mem_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0]
        self.virtual_mem_fieldname = ['total', 'available','percent', 'used', 'free', 'active', 'inactive', 'buffers', 'cached', 'shared', 'slab']
        self.virtual_mem_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0, 0.0]
        self.tempsensor_fieldname = ['current','high', 'critical']
        self.tempsensor_fieldvals = [0.0 , 0.0, 0.0]
        self.samplerThread = threading.Thread(target=self.samplerThread)
        self.samplingInterval = interval
        self.stopThread = False


    def SampleTemp(self):

        tempsensor_name = ""
        # res_temp = psutil.sensors_temperatures()
        res_diskio = psutil.disk_io_counters(perdisk=False, nowrap=True)
        res_swapmem = psutil.swap_memory()
        res_virtmem = psutil.virtual_memory()
        ctr = 0
        global not_firstime
        ts = datetime.datetime.now()
        if (self.not_firstime == False):
            self.temp_dict['time'] = [str(ts)]
        else:
            self.temp_dict['time'] += [str(ts)]

        ## get CPU Load averages
        loadavgs = [x / psutil.cpu_count() * 100 for x in psutil.getloadavg()]
        if (self.not_firstime == False):
            self.temp_dict['cpuloadavg_1min'] = loadavgs[0]
            self.temp_dict['cpuloadavg_5min'] = loadavgs[1]
            self.temp_dict['cpuloadavg_15min'] = loadavgs[2]
        else:
            self.temp_dict['cpuloadavg_1min'] += loadavgs[0]
            self.temp_dict['cpuloadavg_5min'] += loadavgs[1]
            self.temp_dict['cpuloadavg_15min'] += loadavgs[2]


        ## Process the Virtual Memory readings
        ctr = 0
        for val in res_virtmem:
            if (self.not_firstime == False):
                self.temp_dict['vm_'+self.virtual_mem_fieldname[ctr]] = [val]
            else:
                self.temp_dict['vm_'+self.virtual_mem_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the Swap Memory readings
        ctr = 0
        for val in res_swapmem:
            if (self.not_firstime == False):
                self.temp_dict['swap_'+self.swap_mem_fieldname[ctr]] = [val]
            else:
                self.temp_dict['swap_'+self.swap_mem_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the DiskIO readings
        ctr = 0
        for val in res_diskio:
            if (self.not_firstime == False):
                self.temp_dict['diskio_'+self.diskio_fieldname[ctr]] = [val]
            else:
                self.temp_dict['diskio_'+self.diskio_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the Temperature readings
        # for key,values in res_temp.items():
        #     # print (key,' => ',values)
        #     for temp_elems in values:
        #         # print (temp_elems)
        #         tempsensor_name = key
        #         ctr = 0;title_field = True
        #         for val in temp_elems:
        #             if (title_field == True):
        #                 tempsensor_name +=  '-'+str(val)
        #                 title_field = False
        #             else:
        #                 self.tempsensor_fieldvals[ctr] = val
        #                 ctr += 1
                        
                    # print(key,'-', '=>',tempsensor_fieldname[ctr],'-->',sensor_val)
                # print (tempsensor_name,'=>',tempsensor_fieldvals)
                # ctr = 0
                # for elems in self.tempsensor_fieldvals:
                #     # print (tempsensor_name+'-'+tempsensor_fieldname[ctr],'=>',elems)
                #     if (self.not_firstime == False):
                #         self.temp_dict[tempsensor_name+'-'+self.tempsensor_fieldname[ctr]] = [elems]
                #     else:
                #         self.temp_dict[tempsensor_name+'-'+self.tempsensor_fieldname[ctr]] += [elems]
                #     ctr += 1
        self.not_firstime = True

    def getReadings(self):
        return self.temp_dict
    
    def samplerThread(self):
        while (self.stopThread != True):
            self.SampleTemp()
            time.sleep(self.samplingInterval)
        return
    
    def startSampling(self):
        self.stopThread = False
        self.samplerThread.start()
        return 

    def stopSampling(self):
        self.stopThread = True
        self.samplerThread.join()
        return 


In [11]:
import torch.nn.utils.prune as prune
import numpy as np
import copy

import pandas as pd
import io
from torch.profiler import profile, record_function, ProfilerActivity


# COLUMN_NAMES = ['prune-config', 'prune-ratio','vm-percent','swap-percent','cpuloadavg-1min', 'cpuloadavg-5min', 'ratio','memory_consumption','acc']
df_final = pd.DataFrame()
model_name = input("Enter the model name: ")

load_model(model_name)

pruning_method = input("Enter the pruning method: ")

# Define a list of supported pruning techniques
supported_pruning_methods = ['l1_unstructured', 'random_unstructured','ln_structured','random_structured']  # Add more pruning methods as needed

if pruning_method.lower() not in supported_pruning_methods:
    raise ValueError(f"Pruning method '{pruning_method}' is not supported.")


for ratio in np.arange(0.0,1.0,0.1):    
    statsMon = SystemStatsGatherer()
    statsMon.startSampling()
    ratio = round(ratio,2)
    print("Pruning method:", pruning_method)
    print (ratio )
    # Assuming you have a ResNet model defined



    if pruning_method == 'l1_unstructured':
        conv_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                if 'layer' in name:
                    conv_layers_to_prune.append((module, name))
        fc_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                fc_layers_to_prune.append((module, name))

        for layer, name in conv_layers_to_prune:
            prune.l1_unstructured(layer, name='weight', amount=ratio)
        for layer, name in fc_layers_to_prune:
            prune.l1_unstructured(layer, name='weight', amount=ratio)

        for layer, name in conv_layers_to_prune:
            prune.remove(layer, name='weight')

    elif pruning_method == 'random_unstructured':
        fc_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                fc_layers_to_prune.append((module, name))
        conv_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):

                if 'layer' in name:
                    conv_layers_to_prune.append((module, name))


        for layer, name in conv_layers_to_prune:
            prune.random_unstructured(layer, name='weight', amount=ratio)
        for layer, name in fc_layers_to_prune:
            prune.random_unstructured(layer, name='weight', amount=ratio)
        for layer, name in conv_layers_to_prune:
            prune.remove(layer, name='weight')
    elif pruning_method == 'random_structured':
        fc_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                fc_layers_to_prune.append((module, name))
        conv_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                if 'layer' in name:
                    conv_layers_to_prune.append((module, name))
        for layer, name in conv_layers_to_prune:
            prune.random_structured(layer, name='weight', amount=ratio)
        for layer, name in fc_layers_to_prune:
            prune.random_structured(layer, name='weight', amount=ratio)
        for layer, name in conv_layers_to_prune:
            prune.remove(layer, name='weight')
    elif pruning_method == 'ln_structured':
        fc_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                fc_layers_to_prune.append((module, name))
        conv_layers_to_prune = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                if 'layer' in name:
                    conv_layers_to_prune.append((module, name))
        for layer, name in conv_layers_to_prune:
            prune.ln_structured(layer, name='weight', amount=ratio,n=2)
        for layer, name in fc_layers_to_prune:
            prune.ln_structured(layer, name='weight', amount=ratio,n=2)
        for layer, name in conv_layers_to_prune:
            prune.remove(layer, name='weight')
    


    prof = torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA, torch.profiler.ProfilerActivity],profile_memory=True, record_shapes=True,with_flops=True)
    print ('Profiling run completed.\nCompiling result table')
    prof.start()
    acc = __test__(model,model_name)
    prof.stop()

    print ('Profiling run completed.\nCompiling result table')
    prof_result = prof.key_averages().table(sort_by='cpu_memory_usage', row_limit=100)
    prof_result_back = prof_result

    #Trim off the header and footer of the results
    prof_result_back = prof_result_back.rsplit("\n",3)[0]
    prof_result__ = ""
    for line in prof_result_back:
        if "-" not in line.split():
            prof_result__ += line 
    prof_result_back = prof_result__
    # print (prof_result_back)
    df = pd.read_csv(io.StringIO(prof_result_back), sep="\s\s+")

    # pd.set_option('display.max_rows', 500)
    # display(df)

    memory_consumption_str = df[(df['Name']=='[memory]')]['CPU Mem'].iloc[0]
    

    memory_consumption = 0
    if ('Gb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024*1024*1024
    elif ('Mb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024*1024
    elif ('Kb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024
    elif ('b' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])

    print(ratio, memory_consumption, acc)
    statsMon.stopSampling()
    print('Collecting System Stats')
    df_stats = pd.DataFrame.from_dict(statsMon.getReadings())
    agg_df = df_stats[['vm_percent','swap_percent','cpuloadavg_1min', 'cpuloadavg_5min']]
    agg_df__ =  agg_df.mean()
    agg_df__['ratio'] = ratio
    agg_df__['memory_consumption'] = memory_consumption
    agg_df__['acc'] = acc
    agg_df__['prune_config'] = f'{pruning_method}-ratio-{ratio}'
    agg_df__['prune_ratio'] = ratio
   
    
    display(agg_df__)
    df_final = pd.concat([df_final,agg_df__], axis=1)
    print('Aggregated stats')
    display(df_final)
    del prof_result
del model



<>:126: SyntaxWarning: invalid escape sequence '\s'
<>:126: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_2978743/3245841989.py:126: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(io.StringIO(prof_result_back), sep="\s\s+")
STAGE:2024-04-09 20:35:17 2978743:2978743 ActivityProfilerController.cpp:314] Completed Stage: Warm Up

Pruning method: l1_unstructured
0.0
Profiling run completed.
Compiling result table



  8%|▊         | 64/782 [00:16<02:59,  4.00it/s]
/tmp/ipykernel_2978743/3245841989.py:126: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(io.StringIO(prof_result_back), sep="\s\s+")


KeyboardInterrupt: 

In [ ]:
# Define the directory where the CSV files will be saved
SAVE_DIR = BASE_DIR

# Define a function to generate the filename based on model name and pruning method
def generate_filename(model_name, pruning_method):
    # Replace spaces with underscores and concatenate with '.csv' extension
    filename = f"{model_name.replace(' ', '_')}_{pruning_method}.csv"
    # Combine with the directory path
    return SAVE_DIR + filename

# Use the function to generate the filename
DF_SAVEFILENAME = generate_filename(model_name, pruning_method)

# Save the DataFrame to CSV
df_final_transpose = df_final.transpose().reset_index(drop=True)
display(df_final_transpose)
df_final_transpose.to_csv(DF_SAVEFILENAME, encoding='utf-8', sep=',')


,vm_percent,swap_percent,cpuloadavg_1min,cpuloadavg_5min,ratio,memory_consumption,acc,prune_config,prune_ratio
0,67.706135,0.0,35863.9,36641.2,0.0,12.0,80.346,l1_unstructured-ratio-0.0,0.0
1,65.968595,0.0,24983.675,25755.1,0.1,12.0,80.287999,l1_unstructured-ratio-0.1,0.1
2,67.234328,0.0,29278.575,28741.25,0.2,12.0,80.214,l1_unstructured-ratio-0.2,0.2
3,66.967347,0.0,12402.725,17852.325,0.3,12.0,79.597998,l1_unstructured-ratio-0.3,0.3
4,67.07234,0.0,4204.825,9529.9,0.4,12.0,77.089995,l1_unstructured-ratio-0.4,0.4
5,68.015534,0.0,9296.925,8974.275,0.5,12.0,70.694,l1_unstructured-ratio-0.5,0.5
6,68.407692,0.0,11986.975,10632.975,0.6,12.0,44.25,l1_unstructured-ratio-0.6,0.6
7,68.678095,0.0,12636.425,11827.3,0.7,12.0,1.928,l1_unstructured-ratio-0.7,0.7
8,70.363441,0.0,8442.65,10112.9,0.8,12.0,0.1,l1_unstructured-ratio-0.8,0.8
9,72.204255,0.0,4559.25,6927.425,0.9,12.0,0.1,l1_unstructured-ratio-0.9,0.9
